Take a look at https://blog.keras.io/a-ten-minute-introduction-to-sequence-to-sequence-learning-in-keras.html

In [1]:
%matplotlib inline
import matplotlib.pylab as plt
import numpy as np
import pandas as pd
import os
import tensorflow as tf

In [2]:
root = "../data/"
noise=0.0
history = 100 
future = 12
train_batch = 200
test_batch = 200
num_epochs = 5

train = np.load(os.path.join(root, "noisy%s_train.npy" %noise))
vad = np.load(os.path.join(root, "noisy%s_vad.npy" %noise))
test = np.load(os.path.join(root, "noisy%s_test.npy" %noise))

train_steps = train.shape[0] // train_batch
vad_steps = vad.shape[0] // train_batch
test_steps = test.shape[0] // test_batch

In [3]:
train.shape, vad.shape, test.shape

((70350, 224), (9365, 224), (8781, 224))

In [4]:
def make_dataset(data):
    return tf.data.Dataset.from_tensor_slices(data)

def make_iterator(dataset, batch_size, num_epochs):
    return dataset.batch(batch_size).repeat(num_epochs).make_one_shot_iterator().get_next()

def split(data, history, future, batch_size, num_epochs=1):
    xf, xl = data[:, :history], data[:, history:history+future]
    yf, yl = data[:, history+future:2*history+future], data[:, 2*history+future:]
    # add start of sentence to labels (input to the decoder)
    yl_input = np.zeros(shape=(yl.shape[0], yl.shape[1]+1))
    yl_input[:, 1:] = yl
    # add end of sentence to labels (output of the decoder)
    yl_output = np.zeros(shape=(yl.shape[0], yl.shape[1]+1))
    yl_output[:, :-1] = yl
    
    # add new dimension at the end of all arrays
    yf = yf[:, :, np.newaxis]
    yl_input = yl_input[:, :, np.newaxis].astype(np.float32)
    yl_output = yl_output[:, :, np.newaxis].astype(np.float32)
    
    return (yf, yl_input), yl_output

train_gen = split(train, history, future, train_batch, num_epochs)
vad_gen = split(vad, history, future, train_batch, 1)
test_gen = split(test, history, future, test_batch, 1)

In [5]:
train_gen[0][0].shape, train_gen[0][1].shape, train_gen[1].shape

((70350, 100, 1), (70350, 13, 1), (70350, 13, 1))

In [6]:
from tensorflow.python.keras.layers import Dense, LSTM, Input
from tensorflow.python.keras.models import Model
from tensorflow.python.keras.losses import MAE
from tensorflow.python.keras.metrics import MAPE
from tensorflow.python.keras.backend import clear_session

In [7]:
def seq2seq(history, future, latent_dim):
    clear_session()
    # Define an input sequence and process it.
    encoder_inputs = Input(shape=(history, 1))
    encoder = LSTM(latent_dim, return_state=True)
    encoder_outputs, state_h, state_c = encoder(encoder_inputs)
    # We discard `encoder_outputs` and only keep the states.
    encoder_states = [state_h, state_c]

    # Set up the decoder, using `encoder_states` as initial state.
    decoder_inputs = Input(shape=(future+1, 1))
    # We set up our decoder to return full output sequences,
    # and to return internal states as well. We don't use the 
    # return states in the training model, but we will use them in inference.
    decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
    decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                         initial_state=encoder_states)
    decoder_dense = Dense(1, activation='selu')
    decoder_outputs = decoder_dense(decoder_outputs)

    # Define the model that will turn
    # `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
    model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
    return model

In [8]:
m2 = seq2seq(history, future, latent_dim=50)
m2.compile(optimizer='rmsprop', loss=MAE, metrics=[MAPE, MAE])
h2 = m2.fit(x=train_gen[0], y=train_gen[1], batch_size=train_batch, epochs=num_epochs, 
      validation_data=vad_gen)

Train on 70350 samples, validate on 9365 samples
Epoch 1/5
70350/70350 [==============================] - 228s 3ms/step - loss: 0.1168 - mean_absolute_percentage_error: 28422752.0144 - mean_absolute_error: 0.1168 - val_loss: 0.0772 - val_mean_absolute_percentage_error: 18801599.7768 - val_mean_absolute_error: 0.0772
Epoch 2/5
70350/70350 [==============================] - 224s 3ms/step - loss: 0.0666 - mean_absolute_percentage_error: 17078473.5124 - mean_absolute_error: 0.0666 - val_loss: 0.0664 - val_mean_absolute_percentage_error: 10031653.0635 - val_mean_absolute_error: 0.0664
Epoch 3/5
70350/70350 [==============================] - 197s 3ms/step - loss: 0.0553 - mean_absolute_percentage_error: 13029406.1173 - mean_absolute_error: 0.0553 - val_loss: 0.0525 - val_mean_absolute_percentage_error: 14263808.8206 - val_mean_absolute_error: 0.0525
Epoch 4/5
70350/70350 [==============================] - 204s 3ms/step - loss: 0.0477 - mean_absolute_percentage_error: 10461799.7569 - mean_abs

In [9]:
m3 = seq2seq(history, future, latent_dim=50)
m3.compile(optimizer='rmsprop', loss=MAPE, metrics=[MAPE, MAE])
h3 = m3.fit(x=train_gen[0], y=train_gen[1], batch_size=train_batch, epochs=num_epochs, 
      validation_data=vad_gen)

Train on 70350 samples, validate on 9365 samples
Epoch 1/5
70350/70350 [==============================] - 213s 3ms/step - loss: 734721.6606 - mean_absolute_percentage_error: 734721.6606 - mean_absolute_error: 0.4195 - val_loss: 169410.8792 - val_mean_absolute_percentage_error: 169410.8792 - val_mean_absolute_error: 0.4177
Epoch 2/5
70350/70350 [==============================] - 201s 3ms/step - loss: 401224.5604 - mean_absolute_percentage_error: 401224.5604 - mean_absolute_error: 0.4178 - val_loss: 617839.9356 - val_mean_absolute_percentage_error: 617839.9356 - val_mean_absolute_error: 0.4112
Epoch 3/5
70350/70350 [==============================] - 202s 3ms/step - loss: 309001.9542 - mean_absolute_percentage_error: 309001.9542 - mean_absolute_error: 0.4177 - val_loss: 105411.4461 - val_mean_absolute_percentage_error: 105411.4461 - val_mean_absolute_error: 0.4193
Epoch 4/5
70350/70350 [==============================] - 203s 3ms/step - loss: 249273.3744 - mean_absolute_percentage_error: 2

In [10]:
m4 = seq2seq(history, future, latent_dim=100)
m4.compile(optimizer='rmsprop', loss=MAE, metrics=[MAPE, MAE])
h4 = m4.fit(x=train_gen[0], y=train_gen[1], batch_size=train_batch, epochs=num_epochs, 
      validation_data=vad_gen)

Train on 70350 samples, validate on 9365 samples
Epoch 1/5
70350/70350 [==============================] - 407s 6ms/step - loss: 0.1085 - mean_absolute_percentage_error: 26867136.7138 - mean_absolute_error: 0.1085 - val_loss: 0.0746 - val_mean_absolute_percentage_error: 20523189.9221 - val_mean_absolute_error: 0.0746
Epoch 2/5
70350/70350 [==============================] - 401s 6ms/step - loss: 0.0642 - mean_absolute_percentage_error: 19557340.7932 - mean_absolute_error: 0.0642 - val_loss: 0.0657 - val_mean_absolute_percentage_error: 33651767.3134 - val_mean_absolute_error: 0.0657
Epoch 3/5
70350/70350 [==============================] - 401s 6ms/step - loss: 0.0505 - mean_absolute_percentage_error: 13386543.2793 - mean_absolute_error: 0.0505 - val_loss: 0.0428 - val_mean_absolute_percentage_error: 6656499.4861 - val_mean_absolute_error: 0.0428
Epoch 4/5
70350/70350 [==============================] - 398s 6ms/step - loss: 0.0425 - mean_absolute_percentage_error: 7110788.7436 - mean_absol